In [ ]:
import nltk
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from nltk.chunk import conlltags2tree, tree2conlltags
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd

nltk.download('treebank')
nltk.download('universal_tagset')
nltk.download('maxent_ne_chunker')
nltk.download('words')

In [ ]:
class GetFormattedDataset:
    def __init__(self):
        dataset = nltk.corpus.treebank.tagged_sents()
        dataset = [tree2conlltags(nltk.ne_chunk(d)) for d in dataset]
        train_set, test_set = train_test_split(dataset, test_size=0.35, random_state=41)
        train_set, dev_set = train_test_split(dataset, test_size=0.01, random_state=41)
        self.dataset = dict()
        self.dataset["x_train"] = [[d[0].lower() for d in data] for data in train_set]
        self.dataset["y_train"] = [[d[2] for d in data] for data in train_set]
        self.dataset["x_dev"] = [[d[0].lower() for d in data] for data in dev_set]
        self.dataset["y_dev"] = [[d[2] for d in data] for data in dev_set]
        self.dataset["x_test"] = [[d[0].lower() for d in data] for data in test_set]
        self.dataset["y_test"] = [[d[2] for d in data] for data in test_set]
        self.vocab = set([word for sentence in self.dataset["x_train"] for word in sentence])
        self.tags = set([tag for sentence in self.dataset["y_train"] for tag in sentence])

data_dict = GetFormattedDataset()

In [ ]:
# Viterbi Algorithm
class Viterbi:
    def __init__(self, data_dict: GetFormattedDataset):
        self.data_dict = data_dict
        self.START = "<s>"
        self.UNKNOWN = "<unk>"
        self.emission_matrix = {w: {t: 0 for t in self.data_dict.tags} for w in self.data_dict.vocab}
        self.emission_matrix[self.UNKNOWN] = {t: 0 for t in self.data_dict.tags}
        self.transition_matrix = {t1: {t2: 0 for t2 in self.data_dict.tags} for t1 in self.data_dict.tags}
        self.transition_matrix[self.START] = {t: 0 for t in self.data_dict.tags}
        self.tag_count = dict()

    def __fill_tag_count(self):
        self.tag_count = {k: 0 for k in self.data_dict.tags}
        self.tag_count[self.START] = 0
        for sentence_tags in tqdm(self.data_dict.dataset["y_train"], desc="tag count"):
            for tag in sentence_tags:
                self.tag_count[tag] += 1
            self.tag_count[self.START] += 1

    def __fill_emission_matrix(self, smoothing=True):
        for data_idx in tqdm(range(len(self.data_dict.dataset["x_train"])), desc="emission"):
            words = self.data_dict.dataset["x_train"][data_idx]
            tags = self.data_dict.dataset["y_train"][data_idx]
            for word, tag in zip(words, tags):
                self.emission_matrix[word][tag] += 1

        for tag in self.data_dict.tags:
            for word in self.data_dict.vocab:
                if smoothing:
                    self.emission_matrix[word][tag] = (self.emission_matrix[word][tag] + 1) / (self.tag_count[tag] + len(self.data_dict.vocab))
                    self.emission_matrix[self.UNKNOWN][tag] = (0 + 1) / (self.tag_count[tag] + len(self.data_dict.vocab))
                else:
                    self.emission_matrix[word][tag] = self.emission_matrix[word][tag] / self.tag_count[tag]
                    self.emission_matrix[self.UNKNOWN][tag] = 0

    def __fill_transition_matrix(self, smoothing=True):
        for sentence_tags in tqdm(self.data_dict.dataset["y_train"], desc="transition"):
            for i in range(0, len(sentence_tags) - 1):
                tag1 = sentence_tags[i]
                tag2 = sentence_tags[i + 1]
                self.transition_matrix[tag1][tag2] += 1
                if i == 0:
                    self.transition_matrix[self.START][tag1] += 1

        for tag1 in self.transition_matrix.keys():
            for tag2 in self.transition_matrix[tag1].keys():
                if smoothing:
                    self.transition_matrix[tag1][tag2] = (self.transition_matrix[tag1][tag2] + 1) / (self.tag_count[tag1] + len(self.data_dict.tags))
                else:
                    self.transition_matrix[tag1][tag2] = self.transition_matrix[tag1][tag2] / self.tag_count[tag1]

        for tag1 in self.transition_matrix.keys():
            for tag2 in self.transition_matrix[tag1].keys():
                if "I-" in tag1 and "B-" in tag2 and tag1[2:] == tag2[2:]:
                    self.transition_matrix[tag1][tag2] = 0
                if "O" == tag1 and "I-" in tag2:
                    self.transition_matrix[tag1][tag2] = 0
                if "I-" in tag1 and "I-" in tag2 and tag1[2:] != tag2[2:]:
                    self.transition_matrix[tag1][tag2] = 0

    def train(self):
        self.__fill_tag_count()
        self.__fill_transition_matrix()
        self.__fill_emission_matrix()

    def predict(self, sentence: list):  # Viterbi Algorithm
        viterbi = [{t: 0 for t in self.data_dict.tags} for w_idx in range(len(sentence))]
        back_pointer = [{t: 0 for t in self.data_dict.tags} for w_idx in range(len(sentence))]
        for tag in self.data_dict.tags:
            word = sentence[0]
            emission_prob = self.emission_matrix[self.UNKNOWN][tag] if word not in self.emission_matrix else self.emission_matrix[word][tag]
            viterbi[0][tag] = self.transition_matrix[self.START][tag] * emission_prob

        for word_idx in range(1, len(sentence)):
            word = sentence[word_idx]
            prev_word = sentence[word_idx - 1]
            for tag in self.data_dict.tags:
                viterbi[word_idx][tag] = 0
                for prev_tag in self.data_dict.tags:
                    emission_prob = self.emission_matrix[self.UNKNOWN][tag] if word not in self.emission_matrix else self.emission_matrix[word][tag]
                    new_viterbi = viterbi[word_idx - 1][prev_tag] * self.transition_matrix[prev_tag][tag] * emission_prob
                    if new_viterbi > viterbi[word_idx][tag]:
                        viterbi[word_idx][tag] = new_viterbi
                        back_pointer[word_idx][tag] = prev_tag

        last_tag_prob = 0
        for tag in self.data_dict.tags:
            if viterbi[-1][tag] >= last_tag_prob:
                last_tag_prob = viterbi[-1][tag]
                last_tag = tag

        best_tags = [last_tag]

        try:
            for word_idx in range(len(sentence) - 1, 0, -1):
                best_tags.append(back_pointer[word_idx][last_tag])
                last_tag = back_pointer[word_idx][last_tag]
        except Exception as e:
            pass
        return list(reversed(best_tags))

    def evaluate(self):
        tp = 0
        total = 0
        preds, trues = [], []
        for data_idx in tqdm(range(len(self.data_dict.dataset["x_test"]))):
            tags = self.predict(self.data_dict.dataset["x_test"][data_idx])
            for t_pred, t_true in zip(tags, self.data_dict.dataset["y_test"][data_idx]):
                preds.append(t_pred)
                trues.append(t_true)
                total += 1
                if t_pred == t_true:
                    tp += 1
        print(classification_report(trues, preds))
        return tp / total


viterbi = Viterbi(data_dict)
viterbi.train()
viterbi.evaluate()

tag count:   0%|          | 0/3874 [00:00<?, ?it/s]

transition:   0%|          | 0/3874 [00:00<?, ?it/s]

emission:   0%|          | 0/3874 [00:00<?, ?it/s]

  0%|          | 0/1370 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

             0       0.00      0.00      0.00         0
    B-FACILITY       0.75      0.60      0.67        15
         B-GPE       0.92      0.58      0.71       651
         B-GSP       0.00      0.00      0.00        10
    B-LOCATION       0.00      0.00      0.00        10
B-ORGANIZATION       0.84      0.25      0.39       564
      B-PERSON       0.91      0.58      0.71       720
    I-FACILITY       0.73      0.73      0.73        15
         I-GPE       0.90      0.78      0.84        79
    I-LOCATION       0.00      0.00      0.00         8
I-ORGANIZATION       0.75      0.38      0.50       338
      I-PERSON       0.73      0.81      0.77       458
             O       0.97      1.00      0.98     32304

      accuracy                           0.96     35172
     macro avg       0.58      0.44      0.48     35172
  weighted avg       0.96      0.96      0.95     35172



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

0.9590867735698851

In [ ]:
df_dict = dict()
for k in data_dict.tags:
    df_dict[k] = [viterbi.transition_matrix[k][i] for i in data_dict.tags]

pd.DataFrame.from_dict(df_dict, orient='index', columns=data_dict.tags)

,B-FACILITY,I-PERSON,B-PERSON,I-ORGANIZATION,I-GPE,B-GPE,B-LOCATION,B-ORGANIZATION,I-FACILITY,O
B-FACILITY,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.545455,0.090909
I-PERSON,0.021739,0.086957,0.000000,0.000000,0.000000,0.021739,0.021739,0.021739,0.000000,0.717391
B-PERSON,0.006329,0.215190,0.025316,0.006329,0.006329,0.006329,0.006329,0.088608,0.006329,0.632911
I-ORGANIZATION,0.006061,0.000000,0.006061,0.193939,0.000000,0.006061,0.006061,0.000000,0.000000,0.751515
I-GPE,0.011765,0.000000,0.011765,0.000000,0.023529,0.000000,0.011765,0.011765,0.000000,0.882353
B-GPE,0.001603,0.001603,0.001603,0.001603,0.120192,0.003205,0.001603,0.068910,0.001603,0.798077
B-LOCATION,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.076923,0.307692
B-ORGANIZATION,0.002577,0.002577,0.002577,0.322165,0.002577,0.002577,0.002577,0.012887,0.002577,0.646907
I-FACILITY,0.000000,0.000000,0.047619,0.000000,0.000000,0.047619,0.047619,0.047619,0.047619,0.571429
O,0.000155,0.000000,0.001301,0.000000,0.000000,0.006049,0.000048,0.003747,0.000000,0.948968
